## ***Machine Learning - WNBA Playoffs Prediction*** 
This notebook will focus on the preparation of the data. We will be using SQLite to store the data due to its scalability & the fact that it's a relational schema.

https://docs.python.org/3/library/sqlite3.html

Import sqlite3 and connect to database file

### **Imports**

In [30]:
import pandas as pd
import sqlite3
from prep_utils import check_missing_values,parse_columns_types,calculate_summary_statistics

### **Database Connection Setup**

In [31]:
import sqlite3
db = sqlite3.connect("db/ac.db")
db_cur = db.cursor()


**Descriptive Statistics**

Descriptive statistics offer valuable metrics for understanding and summarizing data, assisting in data analysis and decision-making.

In [32]:
column_names = ["coachID","year","tmID","lgID","stint","won","lost","post_wins","post_losses"]
num, non_num = parse_columns_types(db_cur,column_names,"coaches")
calculate_summary_statistics(db_cur,"coaches",num)

+-------------+---------+-----------+-----------------+-------+-------+
| Attribute   |   Count |      Mean |   Std Deviation |   Min |   Max |
+=============+=========+===========+=================+=======+=======+
| year        |     162 |  5.31481  |            8.39 |     1 |    10 |
+-------------+---------+-----------+-----------------+-------+-------+
| stint       |     162 |  0.364198 |            0.48 |     0 |     2 |
+-------------+---------+-----------+-----------------+-------+-------+
| won         |     162 | 14.6728   |           41    |     0 |    28 |
+-------------+---------+-----------+-----------------+-------+-------+
| lost        |     162 | 14.6235   |           32.25 |     2 |    30 |
+-------------+---------+-----------+-----------------+-------+-------+
| post_wins   |     162 |  1.16667  |            3.82 |     0 |     7 |
+-------------+---------+-----------+-----------------+-------+-------+
| post_losses |     162 |  1.17284  |            1.73 |     0 | 

**Checking for missing values (N/A)**

It's normal for large datasets to have missing values, which needs to be handled early on.

In [33]:
check_missing_values(db_cur,column_names,"coaches")

Column 'coachID' has missing values: False
Column 'year' has missing values: False
Column 'tmID' has missing values: False
Column 'lgID' has missing values: False
Column 'stint' has missing values: False
Column 'won' has missing values: False
Column 'lost' has missing values: False
Column 'post_wins' has missing values: False
Column 'post_losses' has missing values: False


In [34]:
#coaches = db_cur.execute("SELECT * FROM coaches")
#coaches.fetchall()

In [35]:
# Get first player from query
players = db_cur.execute("SELECT * FROM players")
players.fetchone()

('abrahta01w',
 'C',
 0,
 0,
 74.0,
 190,
 'George Washington',
 '',
 '1975-09-27',
 '0000-00-00')

In [36]:
#db.close()